In [0]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

In [0]:
# Read file from google drive
df = pd.read_csv(r"drive/My Drive/Colab Notebooks/train.csv", encoding="big5")

FileNotFoundError: ignored

In [0]:
import seaborn as sns
def data_analysis():
    pd.set_option('display.max_columns', None)
    li = list(df.columns.values)[48:73]
    li.append(df.columns.values[-1])
    # plt.hist(df['ANNUAL_PREMIUM_AMT'].fillna(-1), bins=80)
    # plt.show()
    # print(df[li].describe())
    # print(len(df[df["Y1"] == "Y"]))

    # 年繳化保費 50
    # sns.jointplot(df["ANNUAL_PREMIUM_AMT"], df["Y1"])
    plt.hist(df[df["Y1"] == "Y"]['ANNUAL_PREMIUM_AMT'], bins=50, range=(0, 0.002),color="g")
    plt.show()
    print(df[df["Y1"] == "Y"]['ANNUAL_PREMIUM_AMT'].describe())
    plt.hist(df[df["Y1"] == "N"]['ANNUAL_PREMIUM_AMT'], bins=50, range=(0, 0.002))
    plt.show()
    print(df[df["Y1"] == "N"]['ANNUAL_PREMIUM_AMT'].describe())

    # 曾經經手的業務員人數 51
    ax = sns.countplot(df[df["Y1"] == "Y"]["AG_CNT"])
    total = len(df[df["Y1"] == "Y"]["AG_CNT"])
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}'.format(height/total),
                ha="center") 
    plt.show()
    print(df[df["Y1"] == "Y"]["AG_CNT"].describe())

    ax = sns.countplot(df[df["Y1"] == "N"]["AG_CNT"])
    total = len(df[df["Y1"] == "N"]["AG_CNT"])
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}'.format(height/total),
                ha="center") 
    plt.show()
    print(df[df["Y1"] == "N"]["AG_CNT"].describe())


    # 年收入 54
    sns.distplot(df[df["Y1"] == "Y"]["ANNUAL_INCOME_AMT"].fillna(0))
    plt.show()
    print(df[df["Y1"] == "Y"]["ANNUAL_INCOME_AMT"].describe())

    sns.distplot(df[df["Y1"] == "N"]["ANNUAL_INCOME_AMT"].fillna(0))
    plt.show()
    print(df[df["Y1"] == "N"]["ANNUAL_INCOME_AMT"].describe())



    # 58-60 是否訂閱電子報ABC
    # sns.distplot(df[df["Y1"] == "Y"]["A_IND"].fillna(0))
    # plt.show()
    # sns.distplot(df[df["Y1"] == "N"]["A_IND"].fillna(0))
    # plt.show()



    # 62 近一年到C通路申報服務次數

    # 64

    # 69-72 

    # print(df['ANNUAL_INCOME_AMT'].head())
    # plt.hist(df['ANNUAL_INCOME_AMT'], bins=100, range=(0, 0.005))
    # plt.show()
    print(li)
    
    for i in li:
        if (df[i].dtype == np.dtype(np.object)):
            print(i.ljust(30), df[i].unique())
        else:
            print(i.ljust(30), df[i].dtype)
        
    print(pd.get_dummies(df[li]).describe())
    
# data_analysis()

pd.set_option('display.max_columns', None) 
df1 = df[li][:]
print(df1.shape)
print("Y:" + str(len(df1[df1['Y1'] == "Y"])) + " N:" + str(len(df1[df1['Y1'] == "N"])), end="\n\n")
print(df1.dtypes)
print(df1.head(5))

In [0]:
def df_preprocessing(df):
    pd.set_option('display.max_columns', None)
    df1 = df.copy()
    # df1 = df1.sample(10000)
    df1["Y1"].replace({'Y':1, 'N':0}, inplace=True)
    fx = pd.get_dummies(df1.iloc[:, 1:len(df1.columns.values)-1])
    # fx.fillna(0, inplace=True)
    #　fx.fillna(df.mean(), inplace=True)
    x = fx.values
    y = df1.iloc[:, -1].values

    print(x.shape, y.shape)
    # print((pd.get_dummies(df1.iloc[:, 1:len(df1.columns.values)])).head())
    # print(fx.head())
    return x, y

In [0]:
from sklearn import metrics

def validation(clf, x_test, y_test):
    y_predict = clf.predict_proba(x_test)[:, 1]
    print(y_predict)
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC = ", auc)
    plt.plot(fpr, tpr, 'b', label="AUC="+str(round(auc,3)))
    plt.plot([0, 1],[0, 1], 'r--', label="Baseline")

    plt.legend(loc=4)
    plt.show()

In [0]:
import datetime

def read_and_test(file_path, clf):
    tdf = pd.read_csv(file_path, encoding="big5")
    
    fxr = pd.get_dummies(tdf.iloc[:, 1:])
    fxr.fillna(0, inplace=True)
    x_real = fxr.values
    print(x_real.shape)
    y_real_predict = clf.predict_proba(x_real)[:, 1]
    # y_real_predict
    
    tag = datetime.datetime.now().strftime("%y%m%d%H%M%S")

    output = np.array(list(zip(tdf.iloc[:,0].values.astype(str), y_real_predict)))
    # np.savetxt()output

    odf = pd.DataFrame(output, columns=["CUS_ID","Ypred"])
    odf.to_csv(r"drive/My Drive/Colab Notebooks/" + tag + ".csv", index=False)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

if __name__ == '__main__':
    x, y = df_preprocessing(df)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

    print(x_train.shape)
    # print(x_train[:5])

    # clf = SVC(gamma='auto', probability=True)
    clf = XGBClassifier(probability=True)
    clf.fit(x_train, y_train)
    print("------")
    validation(clf, x_test, y_test)
    # read_and_test(r"drive/My Drive/Colab Notebooks/test.csv", clf)